In [46]:
import os
import time
import copy
import torch
import torch.optim as optim
# 引入线性回归常用的损失函数
from torch.nn import Linear, Module, MSELoss
import pprint as pp
import utils.hypergraph_utils as hgut
from models import HGNN
from config import get_config
from datasets import load_feature_construct_H
from datasets import load_ft
import pandas as pd
import numpy as np
import math

In [2]:
# initialize data
data_dir = r"L:\HGNN try1\CG_dataset_rgrs.mat"
fts, lbls, idx_train, idx_test = load_ft(data_dir,'CGNN')

In [3]:
print(fts, lbls, idx_train, idx_test)

[[1.5000e+01 3.6440e+03 0.0000e+00 ... 3.3100e+02 4.0500e+02 9.5000e+01]
 [1.0000e+00 3.4800e+02 0.0000e+00 ... 4.3000e+01 3.2000e+01 1.5000e+01]
 [1.0000e+00 5.5100e+02 0.0000e+00 ... 5.8000e+01 3.7000e+01 4.0000e+00]
 ...
 [8.0000e+00 6.9700e+02 3.0000e+00 ... 4.7000e+01 6.5000e+01 2.2000e+01]
 [8.0000e+00 9.5000e+02 0.0000e+00 ... 7.8000e+01 1.2200e+02 3.2000e+01]
 [1.8000e+01 1.9562e+04 0.0000e+00 ... 1.9200e+03 1.6200e+03 4.0000e+02]] [[16  8 33 ...  8  8 18]] [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 10

In [4]:
# H_dir
H_dir = r"L:\hypergraph prediction\话题用户超图.csv"
W_dir = r"L:\hypergraph prediction\用户超边权值.csv"
H = pd.read_csv(H_dir,header = None)
W = pd.read_csv(W_dir)
H = H.to_numpy()
W = W.iloc[:,1].to_numpy()

In [5]:
G = hgut.generate_G_from_H(H,W, variable_weight=False)

L:\HGNN try1\utils\hypergraph_utils.py:115: RuntimeWarning: divide by zero encountered in power
  DV2 = np.mat(np.diag(np.power(DV, -0.5)))


In [6]:
G = torch.Tensor(G)

In [7]:
G = torch.where(torch.isnan(G), torch.full_like(G, 0), G)
G

tensor([[0.1177, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0435]])

In [8]:
'''
fts, lbls, idx_train, idx_test, H = \
    load_feature_construct_H(data_dir,
                             m_prob=cfg['m_prob'],
                             K_neigs=cfg['K_neigs'],
                             is_probH=cfg['is_probH'],
                             use_mvcnn_feature=cfg['use_mvcnn_feature'],
                             use_gvcnn_feature=cfg['use_gvcnn_feature'],
                             use_mvcnn_feature_for_structure=cfg['use_mvcnn_feature_for_structure'],
                             use_gvcnn_feature_for_structure=cfg['use_gvcnn_feature_for_structure'])
'''

# n_class = int(lbls.max()) + 1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # 指定存放的位置 cpu/gpu

# transform data to device
fts= torch.Tensor(fts).to(device)
lbls = torch.Tensor(lbls).squeeze().long().to(device)
G = torch.Tensor(G).to(device)
idx_train = torch.Tensor(idx_train).long().to(device)
idx_test = torch.Tensor(idx_test).long().to(device)

In [69]:
def train_model(model, criterion,MAE, optimizer, scheduler, num_epochs=25, print_freq=500):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    # 分类任务的评估指标acc
#     best_acc = 0.0
    best_loss = 9999
    best_mrse = 9
    mae = 999
    mrse = 9999
    rmse = 999


    for epoch in range(num_epochs):
        # 间隔输出训练效果
        if epoch % print_freq == 0:
            print('-' * 10)
            print(f'Epoch {epoch}/{num_epochs - 1}')

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']: # 每一轮都训一遍再测一遍
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            idx = idx_train if phase == 'train' else idx_test # 选择相应数据集

            # Iterate over data.
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(fts, G) # model 输出的是n*1
#                 print(outputs[idx].float())
                loss = criterion(outputs[idx].float(), lbls[idx].float()) # 根据自定义的criterion计算训练误差
                
                # _, preds = torch.max(outputs, 1)

                # backward + optimize only if in training phase 在训练阶段进行反向传播和优化
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item() * fts.size(0)
#             print(fts.size(0))
            # running_corrects += torch.sum(preds[idx] == lbls.data[idx])

            epoch_loss = running_loss / len(idx)
#             epoch_acc = running_corrects.double() / len(idx)

            if epoch % print_freq == 0:
                print(f'{phase} Loss: {epoch_loss:.4f}')
            

            # deep copy the model 存储预测效果最好时候的model
            # deep copy 复制完整对象并且存储在独立的内存地址
            # 在评估阶段选择最好的模型，loss最小
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
#                 best_mrse = mrse
                best_model_wts = copy.deepcopy(model.state_dict())
   
                mae = MAE(outputs[idx].float(), lbls[idx].float())
                mrse = MRSE(outputs,lbls,idx).item()
                rmse = math.sqrt(best_loss)
#                 print(f'mrse: {mrse}')
#                 print(f'mae: {mae}')
#                 print(f'rmse: {rmse}')

        if epoch % print_freq == 0:
            print(f'Best val loss: {best_loss:4f}')
            print(f'mrse: {mrse}')
            print(f'mae: {mae}')
            print(f'rmse: {rmse}')
           
            print('-' * 20)

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val loss: {best_loss:4f}')
    print(f'mrse: {mrse}')
    print(f'mae: {mae}')
    print(f'rmse: {rmse}')

    # load best model weights
    model.load_state_dict(best_model_wts) # 读出最好的model并且返回
    return model

In [70]:
def _main():
    '''
    print(f"Classification on {cfg['on_dataset']} dataset!!! class number: {n_class}")
    print(f"use MVCNN feature: {cfg['use_mvcnn_feature']}")
    print(f"use GVCNN feature: {cfg['use_gvcnn_feature']}")
    print(f"use MVCNN feature for structure: {cfg['use_mvcnn_feature_for_structure']}")
    print(f"use GVCNN feature for structure: {cfg['use_gvcnn_feature_for_structure']}")
    print('Configuration -> Start')
    pp.pprint(cfg)
    print('Configuration -> End')
    '''

    # 定义模型，此处分类模型被耦合在HGNN函数中，需要去HGNN那里改预测任务
    model_ft = HGNN(in_ch=fts.shape[1], # 特征矩阵列数
                    n_class=1,
                    n_hid=128,
                    dropout=0)
    model_ft = model_ft.to(device) # 将模型转移到设备上

    # adam优化
    optimizer = optim.Adam(model_ft.parameters(), lr=0.01,weight_decay=0.005)
    # SGD优化
#     optimizer = optim.SGD(model_ft.parameters(), lr=0.01, weight_decay=cfg['weight_decay)'])

    # 动态调整学习率
    # milestone 调整轮次
    # gamma 调整倍数
    # new_lr = lr*gamma when epoch = milestone[i]
    schedular = optim.lr_scheduler.MultiStepLR(optimizer,
                                               milestones=[100,300,500,800],
                                               gamma=0.9)
    # criterion = torch.nn.CrossEntropyLoss() # 分类预测任务中使用了交叉熵损失函数
    # 线性回归要改成
    criterion = torch.nn.MSELoss(reduction = 'mean')
    MAE = torch.nn.L1Loss(reduction='mean')

    model_ft = train_model(model_ft, criterion,MAE, optimizer, schedular, 1500, print_freq=100)

In [72]:
if __name__ == '__main__':
    _main()

----------
Epoch 0/1499
train Loss: 4062.5308
val Loss: 653858.4147
Best val loss: 9999.000000
mrse: 9999
mae: 999
rmse: 999
--------------------
----------
Epoch 100/1499
train Loss: 3597.7955
val Loss: 1967.3193
Best val loss: 1946.487533
mrse: 0.48034271597862244
mae: 10.98414421081543
rmse: 44.11901555287044
--------------------
----------
Epoch 200/1499
train Loss: 3582.7178
val Loss: 1936.3456
Best val loss: 1935.312958
mrse: 0.6953354477882385
mae: 12.140671730041504
rmse: 43.99219200745056
--------------------
----------
Epoch 300/1499
train Loss: 3549.1484
val Loss: 1951.8229
Best val loss: 1787.845213
mrse: 0.4888523519039154
mae: 10.692160606384277
rmse: 42.282918695508705
--------------------
----------
Epoch 400/1499
train Loss: 3514.1502
val Loss: 1797.3037
Best val loss: 1725.049031
mrse: 0.5383505821228027
mae: 10.26809024810791
rmse: 41.53370957430146
--------------------
----------
Epoch 500/1499
train Loss: 3471.5629
val Loss: 1776.0195
Best val loss: 1674.168323
mrs

In [73]:
def MRSE(outputs,lbls,idx):
    sums = 0
    count = len(outputs[idx])
    for i in range(count):
        truth = lbls[idx][i]
        pred = outputs[idx][i]
        delta =((truth-pred)/truth)*((truth-pred)/truth)
        sums = sums+delta
    res = sums / count
    return res

In [74]:
lbls

tensor([16,  8, 33,  ...,  8,  8, 18])

In [77]:
torch.mean(lbls.float())

tensor(20.4512)

In [78]:
torch.std(lbls.float())

tensor(46.9552)